# Importing packages

In [1]:
import pandas as pd 
import numpy as np 
import mlflow
import mlflow.keras
from tensorflow import keras
import tensorflow
from sklearn.metrics import f1_score, confusion_matrix 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv', header=None)
df.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.iloc[:,-1].unique()

array([1, 0])

In [4]:
df.iloc[:,-1].value_counts()

0    500
1    268
Name: 8, dtype: int64

# Data Preprocessing

In [5]:
X = df.iloc[:,:-1]
Y = df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=0, stratify=Y)
mx = MinMaxScaler()

X_train = mx.fit_transform(X_train)
X_test = mx.transform(X_test)


# Setting up experiments

In [6]:
mlflow.set_experiment("Baseline predictions")
mlflow.tensorflow.autolog()

In [8]:
input_ = keras.layers.Input(shape=X_train.shape[-1])
layer1 = keras.layers.Dense(units=6, activation='relu')(input_)
norm = keras.layers.BatchNormalization()(layer1)
output  = keras.layers.Dense(units=1, activation='sigmoid')(norm)
model = keras.Model(input_, output)
model.compile(optimizer= keras.optimizers.Adam(lr=0.001),
             loss = "binary_crossentropy",
             metrics="Accuracy")
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 54        
_________________________________________________________________
batch_normalization (BatchNo (None, 6)                 24        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 85
Trainable params: 73
Non-trainable params: 12
_________________________________________________________________


/opt/conda/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [9]:
with mlflow.start_run(run_name="keras_baseline") as run:
    model.fit(X_train, y_train, epochs=100, validation_split=0.05, shuffle=True)
    preds = model.predict(X_test)
    y_pred = np.where(preds>0.5, 1, 0)
    f1 = f1_score(y_test, y_pred)
    mlflow.log_metric(key="f1_experiement_score", value=f1)
    mlflow.log_artifact("keras_model.ipynb")

Epoch 1/20
16/16 [==============================] - 1s 22ms/step - loss: 0.7087 - accuracy: 0.0000e+00 - val_loss: 0.7174 - val_accuracy: 0.0000e+00
Epoch 2/20
16/16 [==============================] - 0s 3ms/step - loss: 0.6912 - accuracy: 0.0000e+00 - val_loss: 0.7025 - val_accuracy: 0.0000e+00
Epoch 3/20
16/16 [==============================] - 0s 4ms/step - loss: 0.6768 - accuracy: 0.0000e+00 - val_loss: 0.6889 - val_accuracy: 0.0000e+00
Epoch 4/20
16/16 [==============================] - 0s 4ms/step - loss: 0.6526 - accuracy: 0.0000e+00 - val_loss: 0.6784 - val_accuracy: 0.0000e+00
Epoch 5/20
16/16 [==============================] - 0s 4ms/step - loss: 0.6533 - accuracy: 0.0000e+00 - val_loss: 0.6668 - val_accuracy: 0.0000e+00
Epoch 6/20
16/16 [==============================] - 0s 4ms/step - loss: 0.6369 - accuracy: 0.0000e+00 - val_loss: 0.6546 - val_accuracy: 0.0000e+00
Epoch 7/20
16/16 [==============================] - 0s 4ms/step - loss: 0.6295 - accuracy: 0.0000e+00 - val_los